In [1]:
#import packages
import pandas_datareader as pdr
import pandas as pd
import numpy as np
import requests
import datetime

#Create the mapping table to various Fred series
IndexMapping = {
'BAMLC0A1CAAASYTW':{'Index':'ICE BofA AAA US Corporate Index Semi-Annual Yield to Worst','Rating':'AAA','Tenor':'Null','Category':'Credit'},
'BAMLC0A2CAASYTW':{'Index':'ICE BofA AA US Corporate Index Semi-Annual Yield to Worst','Rating':'AA','Tenor':'Null','Category':'Credit'},
'BAMLC0A3CASYTW':{'Index':'ICE BofA Single-A US Corporate Index Semi-Annual Yield to Worst','Rating':'A','Tenor':'Null','Category':'Credit'},
'BAMLC0A4CBBBSYTW':{'Index':'ICE BofA BBB US Corporate Index Semi-Annual Yield to Worst','Rating':'BBB','Tenor':'Null','Category':'Credit'},
'BAMLH0A1HYBBSYTW':{'Index':'ICE BofA BB US High Yield Index Semi-Annual Yield to Worst','Rating':'BB','Tenor':'Null','Category':'Credit'},
'BAMLH0A2HYBSYTW':{'Index':'ICE BofA Single-B US High Yield Index Semi-Annual Yield to Worst','Rating':'B','Tenor':'Null','Category':'Credit'},
'BAMLH0A3HYCSYTW':{'Index':'ICE BofA CCC & Lower US High Yield Index Semi-Annual Yield to Worst','Rating':'CCC & Lower','Tenor':'Null','Category':'Credit'},
'DAAA':{'Index':'Moodys Seasoned Aaa Corporate Bond Yield','Rating':'Aaa','Tenor':'20+ Year','Category':'Credit'},
'DBAA':{'Index':'Moodys Seasoned Baa Corporate Bond Yield','Rating':'Baa','Tenor':'20+ Year','Category':'Credit'},
'AAA10Y':{'Index':'Moodys Seasoned Aaa Corporate Bond Yield Relative to Yield on 10-Year Treasury Constant Maturity','Rating':'Aaa','Tenor':'20+ Year','Category':'Interest Rate Spreads'},
'BAA10Y':{'Index':'Moodys Seasoned Baa Corporate Bond Yield Relative to Yield on 10-Year Treasury Constant Maturity','Rating':'Baa','Tenor':'20+ Year','Category':'Interest Rate Spreads'},
'BAMLC0A0CM':{'Index':'ICE BofA US Corporate Index Option-Adjusted Spread','Rating':'Null','Tenor':'Null','Category':'Credit Spread'},
'BAMLC1A0C13Y':{'Index':'ICE BofA 1-3 Year US Corporate Index Option-Adjusted Spread','Rating':'Null','Tenor':'1-3 Year','Category':'Credit Spread'},
'BAMLC2A0C35Y':{'Index':'ICE BofA 3-5 Year US Corporate Index Option-Adjusted Spread','Rating':'Null','Tenor':'3-5 Year','Category':'Credit Spread'},
'BAMLC3A0C57Y':{'Index':'ICE BofA 5-7 Year US Corporate Index Option-Adjusted Spread','Rating':'Null','Tenor':'5-7 Year','Category':'Credit Spread'},
'BAMLC4A0C710Y':{'Index':'ICE BofA 7-10 Year US Corporate Index Option-Adjusted Spread','Rating':'Null','Tenor':'7-10 Year','Category':'Credit Spread'},
'BAMLC7A0C1015Y':{'Index':'ICE BofA 10-15 Year US Corporate Index Option-Adjusted Spread','Rating':'Null','Tenor':'10-15 Year','Category':'Credit Spread'},
'BAMLC8A0C15PY':{'Index':'ICE BofA 15+ Year US Corporate Index Option-Adjusted Spread','Rating':'Null','Tenor':'15+ Year','Category':'Credit Spread'},
'DGS1MO':{'Index':'Market Yield on U.S. Treasury Securities at 1-Month Constant Maturity','Rating':'Null','Tenor':'1M','Category':'Interest Rates'},
'DGS3MO':{'Index':'Market Yield on U.S. Treasury Securities at 3-Month Constant Maturity','Rating':'Null','Tenor':'3M','Category':'Interest Rates'},
'DGS6MO':{'Index':'Market Yield on U.S. Treasury Securities at 6-Month Constant Maturity','Rating':'Null','Tenor':'6M','Category':'Interest Rates'},
'DGS1':{'Index':'Market Yield on U.S. Treasury Securities at 1-Year Constant Maturity','Rating':'Null','Tenor':'1Y','Category':'Interest Rates'},
'DGS2':{'Index':'Market Yield on U.S. Treasury Securities at 2-Year Constant Maturity','Rating':'Null','Tenor':'2Y','Category':'Interest Rates'},
'DGS3':{'Index':'Market Yield on U.S. Treasury Securities at 3-Year Constant Maturity','Rating':'Null','Tenor':'3Y','Category':'Interest Rates'},
'DGS5':{'Index':'Market Yield on U.S. Treasury Securities at 5-Year Constant Maturity','Rating':'Null','Tenor':'5Y','Category':'Interest Rates'},
'DGS7':{'Index':'Market Yield on U.S. Treasury Securities at 7-Year Constant Maturity','Rating':'Null','Tenor':'7Y','Category':'Interest Rates'},
'DGS10':{'Index':'Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity','Rating':'Null','Tenor':'10Y','Category':'Interest Rates'},
'DGS20':{'Index':'Market Yield on U.S. Treasury Securities at 20-Year Constant Maturity','Rating':'Null','Tenor':'20Y','Category':'Interest Rates'},
'DGS30':{'Index':'Market Yield on U.S. Treasury Securities at 30-Year Constant Maturity','Rating':'Null','Tenor':'30Y','Category':'Interest Rates'},
'IUDSOIA':{'Index':'Daily Sterling Overnight Index Average (SONIA) Rate','Rating':'Null','Tenor':'Null','Category':'Interest Rates'},
'SOFR30DAYAVG':{'Index':'30-Day Average SOFR ','Rating':'Null','Tenor':'1M','Category':'Interest Rates'},
'SOFR90DAYAVG':{'Index':'90-Day Average SOFR','Rating':'Null','Tenor':'3M','Category':'Interest Rates'},
'SOFR180DAYAVG':{'Index':'180-Day Average SOFR','Rating':'Null','Tenor':'6M','Category':'Interest Rates'},
'DEXUSEU':{'Index':'U.S. Dollars to Euro Spot Exchange Rate','Rating':'Null','Tenor':'Null','Category':'Developed Market Foreign Exchange Rates'},
'DEXUSUK':{'Index':'U.S. Dollars to U.K. Pound Sterling Spot Exchange Rate','Rating':'Null','Tenor':'Null','Category':'Developed Market Foreign Exchange Rates'},
'DEXJPUS':{'Index':'Japanese Yen to U.S. Dollar Spot Exchange Rate','Rating':'Null','Tenor':'Null','Category':'Developed Market Foreign Exchange Rates'},
'DEXKOUS':{'Index':'South Korean Won to U.S. Dollar Spot Exchange Rate','Rating':'Null','Tenor':'Null','Category':'Developed Market Foreign Exchange Rates'},
'DEXUSAL':{'Index':'U.S. Dollars to Australian Dollar Spot Exchange Rate','Rating':'Null','Tenor':'Null','Category':'Developed Market Foreign Exchange Rates'},
'DEXCAUS':{'Index':'Canadian Dollars to U.S. Dollar Spot Exchange Rate','Rating':'Null','Tenor':'Null','Category':'Developed Market Foreign Exchange Rates'},
'DEXUSNZ':{'Index':'U.S. Dollars to New Zealand Dollar Spot Exchange Rate','Rating':'Null','Tenor':'Null','Category':'Developed Market Foreign Exchange Rates'},
'DEXCHUS':{'Index':'Chinese Yuan Renminbi to U.S. Dollar Spot Exchange Rate','Rating':'Null','Tenor':'Null','Category':'Emerging Market Foreign Exchange Rates'},
'DEXBZUS':{'Index':'Brazilian Reals to U.S. Dollar Spot Exchange Rate','Rating':'Null','Tenor':'Null','Category':'Emerging Market Foreign Exchange Rates'},
}

#Create a list that contains fred series which defines which indicies we want to pull market data for
FRED_series = ['BAMLC0A1CAAASYTW','BAMLC0A2CAASYTW','BAMLC0A3CASYTW','BAMLC0A4CBBBSYTW','BAMLH0A1HYBBSYTW','BAMLH0A2HYBSYTW','BAMLH0A3HYCSYTW',
'DAAA','DBAA','AAA10Y','BAA10Y','BAMLC0A0CM','BAMLC1A0C13Y','BAMLC2A0C35Y','BAMLC3A0C57Y','BAMLC4A0C710Y','BAMLC7A0C1015Y','BAMLC8A0C15PY',
'DGS1MO','DGS3MO','DGS6MO','DGS1','DGS2','DGS3','DGS5','DGS7','DGS10','DGS20','DGS30','IUDSOIA','SOFR30DAYAVG','SOFR90DAYAVG','SOFR180DAYAVG',
'DEXUSEU','DEXUSUK','DEXJPUS','DEXKOUS','DEXUSAL','DEXCAUS','DEXUSNZ','DEXCHUS','DEXBZUS']

#Create a list for rate curve steepner series 

derived_series = ['3_MO_MINUS_2Y_SPREAD','2_YR_MINUS_5YR_SPREAD','2_YR_MINUS_10YR_SPREAD','5_YR_MINUS_10YR_SPREAD','10_YR_MINUS_30YR_SPREAD']

#add the two series together
data_series = FRED_series + derived_series

#define start and end dates
start = datetime.datetime(2021, 1, 1)
end = datetime.datetime.now()

#create session
session = requests.Session()

def get_fred_data(param_list, start_date, end_date):
  df = pdr.DataReader(param_list, 'fred', start_date, end_date,session=session)
  return df.reset_index()

data = get_fred_data(param_list=FRED_series, 
                   start_date=start, 
                   end_date=end)

### Once we have all the data series we want to pull from FRED, we need to perform a number of transformations for our data to be plotted correctly.

These include:
- Adding the rate curve slope across different tenors
- Fill null values with previous date if it exists
- Creating a column for Yield (in Bps) for reference rates and corporate bond indices
- Times Series Changes (1 Week, 1 Month, 3 Month, 6 Month)
- Replace any infinity values and round all numerical values to 4 decimal points

In [ ]:
#Create Rate Curve Steepners
data['3_MO_MINUS_2Y_SPREAD']=data['DGS3MO']-data['DGS2']
data['2_YR_MINUS_5YR_SPREAD']=data['DGS2']-data['DGS5']
data['2_YR_MINUS_10YR_SPREAD']=data['DGS2']-data['DGS10']
data['5_YR_MINUS_10YR_SPREAD']=data['DGS5']-data['DGS10']
data['10_YR_MINUS_30YR_SPREAD']=data['DGS10']-data['DGS30']

#ffill null values with most recent value to get around missing or unavailable data points
data.fillna(method='ffill',inplace=True)

#Melt into dataframe
df = pd.melt(data,id_vars=['DATE'], value_vars=data_series,var_name='Index Identifier',value_name='Yield (%)').rename(columns={'DATE':'Date'})

#Map series dictionary 
df['Index']=df['Index Identifier'].map(IndexMapping).str['Index']
df['Rating']=df['Index Identifier'].map(IndexMapping).str['Rating']
df['Tenor']=df['Index Identifier'].map(IndexMapping).str['Tenor']
df['Category']=df['Index Identifier'].map(IndexMapping).str['Category']

#define which indices we want to see the yield in Bps
IndexList_BpsConversion = [
'BAMLC0A1CAAASYTW','BAMLC0A2CAASYTW','BAMLC0A3CASYTW','BAMLC0A4CBBBSYTW','BAMLH0A1HYBBSYTW','BAMLH0A2HYBSYTW','BAMLH0A3HYCSYTW',
'DAAA','DBAA','AAA10Y','BAA10Y','BAMLC0A0CM','BAMLC1A0C13Y','BAMLC2A0C35Y','BAMLC3A0C57Y','BAMLC4A0C710Y','BAMLC7A0C1015Y','BAMLC8A0C15PY',
'DGS1MO','DGS3MO','DGS6MO','DGS1','DGS2','DGS3','DGS5','DGS7','DGS10','DGS20','DGS30','IUDSOIA','SOFR30DAYAVG','SOFR90DAYAVG',
'SOFR180DAYAVG','3_MO_MINUS_2Y_SPREAD','2_YR_MINUS_5YR_SPREAD','2_YR_MINUS_10YR_SPREAD','5_YR_MINUS_10YR_SPREAD','10_YR_MINUS_30YR_SPREAD']

#convert to bps
def BpsMultiplier(Index_Identifier):
    """
    Function to apply bps conversion if the index is within the IndexList_BpsConversion list.
    
    Parameters
    ----------
    Index Identifier : pandas dataframe
        List which holds series that need the Yield (%) column to be converted to Yield in Bps
        
    """
    if Index_Identifier in IndexList_BpsConversion:
        return 100
    else:
        return 1

#create new column for Yield in Bps
df['Value Multiplier']=df['Index Identifier'].map(BpsMultiplier)
df['Yield (Bps)']=df['Value Multiplier']*df['Yield (%)']

#Create new columns for Times Series Changes

df['1_week_ago']=df.groupby('Index Identifier')['Yield (%)'].shift(5)
df['1_week_ago_change'] = df['Yield (%)'].diff(periods=5)
df['1_week_ago_change_pct'] = df.groupby('Index Identifier')['Yield (%)'].apply(lambda x: x.pct_change(periods=5))

df['1_month_ago']=df.groupby('Index Identifier')['Yield (%)'].shift(20)
df['1_month_ago_change'] = df['Yield (%)'].diff(periods=20)
df['1_month_ago_change_pct'] = df.groupby('Index Identifier')['Yield (%)'].apply(lambda x: x.pct_change(periods=20))

df['3_month_ago']=df.groupby('Index Identifier')['Yield (%)'].shift(60)
df['3_month_ago_change'] = df['Yield (%)'].diff(periods=60)
df['3_month_ago_change_pct'] = df.groupby('Index Identifier')['Yield (%)'].apply(lambda x: x.pct_change(periods=60))

df['6_month_ago']=df.groupby('Index Identifier')['Yield (%)'].shift(120)
df['6_month_ago_change'] = df['Yield (%)'].diff(periods=120)
df['6_month_ago_change_pct'] = df.groupby('Index Identifier')['Yield (%)'].apply(lambda x: x.pct_change(periods=120))

#Replace any infinity values and round all numerical values to 4 decimal points
df.replace([np.inf, -np.inf], 0, inplace=True)
df = df.round(decimals=4)

### Export the dataframe as a excel file and have PowerBI point to the file as a datasource

In [ ]:
'''
define columns needed and export to CSV -> remember to change the file path
'''

df[['Date', 'Index Identifier', 'Yield (%)', 'Index', 'Rating', 'Tenor',
       'Category', 'Value Multiplier', 'Yield (Bps)', '1_week_ago',
       '1_week_ago_change', '1_week_ago_change_pct', '1_month_ago',
       '1_month_ago_change', '1_month_ago_change_pct','3_month_ago',
       '3_month_ago_change', '3_month_ago_change_pct', '6_month_ago',
       '6_month_ago_change', '6_month_ago_change_pct']].to_csv('\\\\user\\downloads\\Python\\Market_Data.csv',index=False)